In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pandas.io.sql as sqlio

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

## load Csv data

In [3]:
#import WNinitCsv as csv
#con = psycopg2.connect(database="mid_term_project", user="lhl_student", password="lhl_student", host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com", port="5432")
#cur = con.cursor()
#csv.WNinitCsv()

## Clean Data

In [4]:
WNflight = pd.read_csv('../../../data/flights/flights_by_top_origin.csv')


In [5]:
delayed = WNflight.loc[WNflight['arr_delay'] >= 15]
early = WNflight.loc[WNflight['arr_delay'] <= 0]

In [6]:
early.value_counts(['origin_city_name', 'mkt_unique_carrier'])

origin_city_name       mkt_unique_carrier
Atlanta, GA            DL                    429337
Dallas/Fort Worth, TX  AA                    282665
Charlotte, NC          AA                    262901
Chicago, IL            UA                    233433
Houston, TX            UA                    190200
                                              ...  
Charlotte, NC          NK                       344
Denver, CO             G4                       179
Chicago, IL            VX                       126
Denver, CO             VX                       110
Houston, TX            B6                        64
Length: 86, dtype: int64

In [7]:
delayed.value_counts(['origin_city_name', 'mkt_unique_carrier'])

origin_city_name       mkt_unique_carrier
Dallas/Fort Worth, TX  AA                    116895
Chicago, IL            UA                     94849
Charlotte, NC          AA                     93712
Atlanta, GA            DL                     92059
Chicago, IL            AA                     79998
                                              ...  
Denver, CO             G4                       121
Phoenix, AZ            HA                       116
Chicago, IL            VX                        47
Houston, TX            B6                        44
Denver, CO             VX                        41
Length: 86, dtype: int64

In [8]:
#We don't need them since same as mkt_carrier
WNflight.drop(['mkt_carrier_fl_num','op_unique_carrier','tail_num','branded_code_share'],axis=1,inplace=True)

In [9]:
#IN names
WNflights02 = WNflight.drop(['origin','dest'],axis=1)

In [10]:
#do not want cancelled flights
WNflights02 = WNflights02[WNflights02['cancelled'] == 0.0]

In [11]:
WNflights03 = WNflights02.copy()

In [12]:
#Cancelled useless for training
WNflights03.drop(['cancelled'],axis=1,inplace=True)

In [13]:
WNflights03[WNflights03['dup'] == "Y"]

,fl_date,mkt_unique_carrier,mkt_carrier,op_carrier_fl_num,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name


In [14]:
## dup has no Y value
WNflights03.drop(['dup'],axis =1, inplace=True)

In [15]:
WNflights03 = WNflights03[WNflights03['arr_delay'].notnull()]

In [16]:
## all mostly empty or repeated data
WNflights03.drop(['no_name','longest_add_gtime','total_add_gtime','first_dep_time'],axis=1, inplace=True)

In [17]:
## only 1 for flights
WNflights03.drop(['flights'],axis=1, inplace=True)

In [18]:
#repeated info
WNflights03.drop(['dep_time','arr_time'],axis=1, inplace=True)

In [19]:
## all the same
WNflights03.drop(['diverted'],axis=1, inplace=True)

In [20]:
#useless
WNflights03.drop(['cancellation_code'],axis=1, inplace=True)

In [21]:
WNflights03.sort_values(['fl_date'])


,fl_date,mkt_unique_carrier,mkt_carrier,op_carrier_fl_num,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
4448406,2018-01-01,UA,UA,5428,13930,"Chicago, IL",10693,"Nashville, TN",1928,25.0,16.0,2009.0,2109.0,6.0,2058,17.0,90.0,82.0,60.0,409,0.0,5.0,0.0,0.0,12.0
4447707,2018-01-01,AA,AA,3510,11298,"Dallas/Fort Worth, TX",13342,"Milwaukee, WI",2030,7.0,18.0,2055.0,2247.0,7.0,2249,5.0,139.0,137.0,112.0,853,NaN,NaN,NaN,NaN,NaN
4447706,2018-01-01,AA,AA,3509,13930,"Chicago, IL",14730,"Louisville, KY",1825,0.0,19.0,1844.0,2032.0,6.0,2045,-7.0,80.0,73.0,48.0,286,NaN,NaN,NaN,NaN,NaN
4447705,2018-01-01,AA,AA,3507,11298,"Dallas/Fort Worth, TX",15624,"Valparaiso, FL",1445,26.0,30.0,1541.0,1703.0,8.0,1638,33.0,113.0,120.0,82.0,641,26.0,0.0,7.0,0.0,0.0
4447704,2018-01-01,AA,AA,3504,11298,"Dallas/Fort Worth, TX",10185,"Alexandria, LA",845,20.0,24.0,929.0,1013.0,4.0,955,22.0,70.0,72.0,44.0,285,4.0,0.0,2.0,0.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077755,2019-12-31,NK,NK,1168,14107,"Phoenix, AZ",13930,"Chicago, IL",1920,-8.0,9.0,1921.0,2313.0,5.0,2342,-24.0,202.0,186.0,172.0,1440,NaN,NaN,NaN,NaN,NaN
2077754,2019-12-31,NK,NK,1167,13930,"Chicago, IL",14107,"Phoenix, AZ",1525,-4.0,12.0,1533.0,1750.0,4.0,1826,-32.0,241.0,213.0,197.0,1440,NaN,NaN,NaN,NaN,NaN
2077753,2019-12-31,NK,NK,1165,10397,"Atlanta, GA",11618,"Newark, NJ",1702,21.0,11.0,1734.0,1923.0,7.0,1920,10.0,138.0,127.0,109.0,746,NaN,NaN,NaN,NaN,NaN
2077727,2019-12-31,AA,AA,4235,11298,"Dallas/Fort Worth, TX",12511,"Joplin, MO",1429,-4.0,13.0,1438.0,1524.0,16.0,1550,-10.0,81.0,75.0,46.0,327,NaN,NaN,NaN,NaN,NaN


In [22]:
# Convert the date to datetime64
WNflights03['fl_date'] = pd.to_datetime(WNflights03['fl_date'], format='%Y-%m-%d')
  
WNflights03.loc[(WNflights03['fl_date'].dt.month == 1) & (WNflights03['origin_airport_id'] == 11057)].value_counts('mkt_carrier')

mkt_carrier
AA    35419
DL     1704
UA     1102
WN      481
B6      223
F9      192
dtype: int64

In [23]:
model_df = WNflights03.copy()
model_df = model_df.loc[model_df['mkt_unique_carrier'].isin(['WN', 'AA', 'UA', 'DL', 'B6', 'NK', 'G4', 'AS', 'F9', 'HA', 'VX'])]
model_df['year'] = model_df['fl_date'].dt.year
model_df['month'] = model_df['fl_date'].dt.month
model_df = model_df[['fl_date',  
       'origin_airport_id', 'dest_airport_id','mkt_unique_carrier', 'month', 'year',
       'taxi_out', 'wheels_off',
       'wheels_on', 'taxi_in','distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay','arr_delay']]

In [24]:
#
model_df.loc[(model_df['fl_date'].dt.month == 1) & (model_df['dest_airport_id'] == 10423)].sort_values(['origin_airport_id', 'mkt_unique_carrier', 'fl_date','dest_airport_id'])

,fl_date,origin_airport_id,dest_airport_id,mkt_unique_carrier,month,year,taxi_out,wheels_off,wheels_on,taxi_in,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,arr_delay
4449171,2018-01-01,10397,10423,DL,1,2018,13.0,1227.0,1335.0,4.0,813,NaN,NaN,NaN,NaN,NaN,-11.0
4449276,2018-01-01,10397,10423,DL,1,2018,25.0,1647.0,1747.0,7.0,813,NaN,NaN,NaN,NaN,NaN,-1.0
4449304,2018-01-01,10397,10423,DL,1,2018,12.0,1937.0,2041.0,3.0,813,NaN,NaN,NaN,NaN,NaN,-23.0
4449490,2018-01-01,10397,10423,DL,1,2018,18.0,2223.0,2320.0,14.0,813,NaN,NaN,NaN,NaN,NaN,2.0
4449494,2018-01-01,10397,10423,DL,1,2018,14.0,853.0,959.0,4.0,813,NaN,NaN,NaN,NaN,NaN,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474306,2019-01-27,14771,10423,WN,1,2019,16.0,857.0,1352.0,3.0,1504,NaN,NaN,NaN,NaN,NaN,-25.0
2485101,2019-01-28,14771,10423,WN,1,2019,15.0,701.0,1145.0,19.0,1504,NaN,NaN,NaN,NaN,NaN,-21.0
2489453,2019-01-29,14771,10423,WN,1,2019,10.0,654.0,1146.0,11.0,1504,NaN,NaN,NaN,NaN,NaN,-28.0
2492815,2019-01-30,14771,10423,WN,1,2019,27.0,712.0,1208.0,4.0,1504,NaN,NaN,NaN,NaN,NaN,-13.0


### Combine this DF with passengers

In [25]:
model_df.to_csv('../../../data/flights/newflightTrain.csv')

KeyboardInterrupt: 

## Relationship Tests

In [ ]:
WNflights03['flight_status'] = (WNflights03['arr_delay'] > 15).replace([True,False],[1,0])

In [ ]:

plt.figure(figsize=(20, 10))
sns.set(font_scale=1.6)
axis = sns.countplot(x=WNflights03['mkt_carrier'], data=WNflights03, order=WNflights03['mkt_carrier'].value_counts().iloc[0:18].index, orient="v")
axis.set_xticklabels(axis.get_xticklabels(), rotation=90, ha='right')
plt.title('TOTAL NUMBER OF FLIGHTS BY AIRLINE', fontsize=24)
plt.xlabel('OP_CARRIER/AIRLINE', fontsize=18)
plt.ylabel('NUMBER OF FLIGHTS', fontsize=18)
plt.tight_layout()
plt.show()

In [ ]:

plt.figure(figsize=(20, 10))
WNflights03.groupby('mkt_carrier').flight_status.sum().sort_values(ascending=False).plot.bar()
plt.title('TOTAL DELAYED FLIGHTS BY AIRLINE', fontsize=20)
plt.xlabel('AIRLINEE NAME', fontsize=16)
plt.ylabel('NUMBRE OF FLIGHTS', fontsize=16)
plt.rc('xtick',labelsize=10)
plt.rc('ytick',labelsize=10)
plt.show()

In [ ]:
test_1 = WNflights03[['mkt_carrier', 'flight_status']].groupby(['mkt_carrier']).sum().reset_index()
test_2 = WNflights03.mkt_carrier.value_counts().rename_axis('mkt_carrier').reset_index(name='TOTAL_FLIGHTS')
overview = pd.merge(test_1, test_2, on='mkt_carrier')
overview['carrier_perc_delay'] = round((overview['flight_status']*100)/overview['TOTAL_FLIGHTS'], 2)
overview.rename({'FLIGHT_STATUS': 'DELAYED_FLIGHTS'}, axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(20, 10))
overview.groupby('mkt_carrier').carrier_perc_delay.sum().sort_values(ascending=False).plot.bar(fontsize=14)
plt.title('PERCENTAGE OF DELAYED FLIGHTS BY AIRLINE', fontsize=20)
plt.xlabel('AIRLINE', fontsize=16)
plt.ylabel('PERCENTAGE', fontsize=16)
plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)
plt.show()

In [ ]:
test_3 = WNflights03[['mkt_carrier', 'arr_delay']]
test_3 = test_3.groupby(['mkt_carrier']).mean().reset_index()
test_3.rename({'arr_delay': 'carrier_delay'}, axis=1, inplace=True)
test_3 = test_3.sort_values(['carrier_delay']).reset_index(drop=True)

In [ ]:
fig = plt.figure(1, figsize=(15,9))
sns.set(font_scale=1.6)
ax = sns.barplot(x='carrier_delay', y='mkt_carrier', data=test_3)
ax.set_title('AVERAGE ARRIVAL DELAY BY AIRLINE (mins)', fontsize=18)
ax.set_xlabel('AVERAGE DELAY BY AIRLINE (min)', fontsize=14)
plt.show()

## Some more cleaning

In [26]:
WNflights04 = WNflights03.copy()

In [27]:
WNflights04.drop(['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay'], axis=1, inplace=True)

In [28]:
WNflights04.drop(['air_time'], axis=1,inplace=True)

In [29]:
WNflights04.drop(['actual_elapsed_time'], axis=1,inplace=True)

In [30]:
WNflights04.drop(['taxi_out','wheels_off','taxi_in','wheels_on'], axis=1,inplace=True)

## Some more Tests

In [31]:
test04 = WNflights04.groupby('op_carrier_fl_num').mean()['arr_delay']
test04 = pd.DataFrame({'fl_num':test04.index,'fl_delay':test04.values})
test04.sort_values('fl_delay',ascending=False)

,fl_num,fl_delay
6973,9341,1110.000000
6999,9389,720.500000
6926,7306,492.000000
6994,9383,337.000000
6996,9386,309.214286
...,...,...
6943,7678,-23.000000
6919,6998,-27.000000
6940,7460,-28.000000
6933,7340,-28.000000


In [32]:
test05 = WNflights04.groupby('tail_num').mean()['arr_delay']
test05 = pd.DataFrame({'tail_num':test05.index,'tail_delay':test05.values})
test05.sort_values('tail_delay',ascending=False)

KeyError: 'tail_num'

In [33]:
dates = WNflights04['fl_date'].values.tolist()

In [34]:
month = []
day = []
year = []
for i in dates:
    year.append(i[0:4])
    month.append(i[5:7])
    day.append(i[8:10])

TypeError: 'int' object is not subscriptable

In [35]:
WNflights04['day'] = day
WNflights04['month'] = month
WNflights04['year'] = year

ValueError: Length of values (0) does not match length of index (5071774)

In [40]:
test07 = WNflights04[['month','arr_delay']]
test07 = test07.groupby(['month'],as_index=False).mean()
test07 = test07.rename({'arr_delay':'month_delay'},axis=1)
test07 = test07.sort_values('month_delay',ascending=False)

KeyError: "['month'] not in index"

In [39]:
test07

NameError: name 'test07' is not defined

In [ ]:
plt.figure(figsize=(20,10))
test07.plot.bar(fontsize=13)
plt.title('Month Delay',fontsize=20)
plt.xlabel('Month', fontsize=16)
plt.ylabel('Delay', fontsize=16)
plt.rc('xtick',labelsize=10)
plt.rc('ytick',labelsize=10)
plt.show()

In [ ]:
#test08 = WNflights04.groupby(['origin_airport_id','dest_airport_id','mkt_carrier']).mean()['arr_delay']

In [ ]:
#test08.index

In [37]:
flightFinal = WNflights04.copy()

In [38]:
flightFinal = flightFinal.merge(test07,on='month')

NameError: name 'test07' is not defined

In [ ]:
test_3.head()

In [ ]:
flightFinal = flightFinal.merge(test_3,on='mkt_carrier')

In [1]:
overview = overview.drop(['flight_status'], axis = 1)

NameError: name 'overview' is not defined

In [ ]:
flightFinal = flightFinal.merge(overview,on='mkt_carrier')

In [ ]:
flightFinal.drop(['flight_status','dep_delay'],axis=1,inplace=True)

In [ ]:
flightFinalTrain = flightFinal.drop(['fl_date','origin_city_name','dest_city_name'],axis=1)

In [ ]:
flightFinal.to_csv('../../../data/flights/flightFinal.csv')

In [ ]:
flightFinalTrain.to_csv('../../../data/flights/flightFinalTrain.csv')

In [ ]:
flightFinal.head()

In [ ]:
# Turn the categories into numbers
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["mkt_carrier", "tail_num"]
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot",
                                   one_hot,
                                   categorical_features)],
                                   remainder="passthrough")

transformed_X = transformer.fit_transform(flightFinalTrain)
transformed_X